In [1]:
import requests 
from bs4 import BeautifulSoup
import re
import pandas as pd
import time as timetime
from selenium import webdriver
import urllib
from tqdm.notebook import tqdm_notebook

In [5]:
from selenium import webdriver

In [24]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [9]:
## 청정원레시피
def get_recipe1(have_list) :
    recipe = []
    url_list=[]
    
    ## 사이트안에서 가진 재료를 입력했을때의 레시피 리스트 접속
    for i, val in enumerate(have_list) :
        if i == 0 :
            url_list.append(val)
        else :
            a = "+"
            b = a + val
            url_list.append(b)
    url_str = ''.join(url_list)
    resp = requests.request("GET", "http://www.chungjungone.com/knowhow/recipe/recipeListNew.do?s_str={}&recp_kind_cd=&recp_kind_cd_nm=&recp_situ_cd=&recp_situ_cd_nm=&x=0&y=0".format(url_str)) 
    soup = BeautifulSoup(resp.text)
    find_ul = soup.find('ul', class_ = 'recipe_ul fourList')
    
    ## 관련 레시피리스트가 없을 경우
    if find_ul is None :
        recipe.append({
                'title' : "",
                'url' : "",
                'time' : "",
                'level' : "",
                'ingredient' : ""
            })
        return recipe
    else :
        find_li = find_ul.find_all('li')

    ## url 가져오기
    recipe_url = []
    for i in find_li :
        find_a = i.find('a')
        url = find_a.get('onclick')
        url = str(url)
        url_ = re.findall("\d+", url)
        recipe_url.append(url_[-1])

    ## title 가져오기
    for i in recipe_url :
        try:
            url_num = i
            resp = requests.request("GET", 'http://www.chungjungone.com/knowhow/recipe/recipeView1New.do?s_str={}&recp_kind_cd=&recp_kind_cd_nm=&recp_situ_cd=&recp_situ_cd_nm=&x=0&y=0&recp_seq={}'.format(url_str,url_num)) 
            soup = BeautifulSoup(resp.text)
            find_div = soup.find('div', class_ = 'editRecipy')
            find_h2 = find_div.find('h2')
            title = find_h2.text

            find_ul = soup.find('ul', class_ = 'reInfo')
            span_list = []
            for i in find_ul :
                find_span = i.find('span')
                span_list.append(find_span)

            ## keyword 가져오기(몇분/난이도)
            keyword = []
            for i in span_list[1:8:2] :
                i = str(i)
                i = i[6:]
                i = i.split('<')
                data = i[0]
                if '조리' in data:
                    i = data[3:]
                    keyword.append(i)
                else :
                    keyword.append(data)

            for i in keyword :
                if "분" in i :
                    if '인' not in i :
                        time = i
                elif ("급") in i :
                    level = i
                else :
                    pass

            # 재료 가져오기

            source = []
            find_div = soup.find('div', class_ = 'texAlCenter')
            find_div = str(find_div)
            find_div = find_div.replace('<br/>', ',').replace('</div>', '').replace('<div class="texAlCenter">', '').replace(':', ',').replace(',,', ',')
            source.append(find_div)

            list_ = source[0].split(',')
            for i in list_ :
                if '소스' in i :
                    list_.remove(i)
            ingredient = []
            for i in list_ :
                i = i.strip()
                if len(i) == 0 :
                    pass
                else : 
                    ingredient.append(i)
                    
            recipe.append({
                'title' : title,
                'url' : 'http://www.chungjungone.com/knowhow/recipe/recipeView1New.do?s_str={}&recp_kind_cd=&recp_kind_cd_nm=&recp_situ_cd=&recp_situ_cd_nm=&x=0&y=0&recp_seq={}'.format(url_str,url_num),
                'time' : time,
                'level' : level,
                'ingredient' : ingredient
            })

        except :
            continue

    return recipe


##만개의 레시피
def get_recipe2(have_list):
    
    base_url='https://www.10000recipe.com/recipe/list.html'

    have_list1 = "+".join(have_list) 
    food = " ".join(have_list)
    
    params={
        'q':food
    }
    
    resp=requests.get(base_url,params=params)
    soup=BeautifulSoup(resp.text)    
    
    
    url_list=[]
    title_list=[]
    result_list=[]
    
    recipe_list_tags=soup.select('ul.common_sp_list_ul.ea4')
    li_tags=recipe_list_tags[0].find_all('li')

    
    for li in tqdm_notebook(li_tags):
        anchor_tag=li.find('a')
        href=anchor_tag.get('href')
        title_tag=li.find_all('div',class_='common_sp_caption_tit line2')
        
        result_list.append({'title':title_tag,
                           'url':href})
        
        url_list.append(href)
        for title in title_tag:
            title_list.append(title.text.strip())
    
    
    url_list2=[]
    for url in url_list:
        each_url='https://www.10000recipe.com'+url
        url_list2.append(each_url)
        
    span1=[]
    span2=[]
    span3=[]
    for url in tqdm_notebook(url_list):
        each_url='https://www.10000recipe.com'+url

        resp=requests.get(each_url)
        soup=BeautifulSoup(resp.text)

        desc_list_tags=soup.select('div.view2_summary_info')

        for desc in desc_list_tags:
            span_tag=desc.select('span')
            if len(span_tag) == 3:
                time = span_tag[1].text
                lev = span_tag[2].text
                span2.append(time)
                span3.append(lev)
            
            elif not span_tag:
                span2.append(None)
                span3.append(None)

            else:
                time=None
                lev = span_tag[0]
                span2.append(time)
                span3.append(lev)

    ing_list=[]
    li_list=[]
    for url in url_list:
        each_url='https://www.10000recipe.com'+url

        resp=requests.get(each_url)
        soup=BeautifulSoup(resp.text)

        ing_list_tags=soup.select('div.ready_ingre3')
        cont_tags=soup.select('div.cont_ingre')

        for i in ing_list_tags:
            li_tags=i.find_all('li')
            li_list.append(li_tags)

        for j in cont_tags:
            dd_tags=j.find_all('dd')
            li_list.append(dd_tags)
            
    ingr_list=[]
    for i in tqdm_notebook(li_list):
        temp = []
        for j in i:
            all_ing=j.text.replace(' ','').strip().split('\n')
            all_ing = " ".join(all_ing)
            temp.append(all_ing)
        ingr_list.append(temp) 
        temp=[]
        
    df=pd.DataFrame({'title':title_list,
                     'url':url_list2,
                     'time':span2,
                     'level':span3,
                     'ingredient':ingr_list})
    
    return df

## 해먹남녀
def get_recipe3(have_list,page_num=2):
    have_list1 = "+".join(have_list) ## "양파 가지"
    have_list2 = " ".join(have_list)
    url = 'https://www.haemukja.com/recipes?utf8=%E2%9C%93&sort=rlv&name={}'.format(have_list1)
    result_list = []
    
    for i in range(page_num):
        params = {
            'name': have_list2,
            'page': 1,
            'sort': 'rlv'
         }

        resp = requests.get(url, params = params)
        resp

        soup = BeautifulSoup(resp.text)

        titles = soup.select('ul.lst_recipe>li>p>a')

        for a_tag in titles:

            recipe_url = 'https://www.haemukja.com'+ a_tag.get('href')
            detail_resp = requests.get(recipe_url)
            html = BeautifulSoup(detail_resp.text)

            prd_time = str(html.select('div.top>dl>dd'))
            prd_time = re.sub('<.+?>', '', prd_time, 0).strip()
            prd_sources = str(html.select('div.btm>ul>li'))
            prd_sources=re.sub('<.+?>', '', prd_sources,0)

            result_list.append({
                'title': (a_tag.text.split('\n')[1]).strip(),
                'url': recipe_url,
                'time':prd_time.replace('[','').replace(']','').split(',')[0],
                'level': 'null',
                'ingredient': prd_sources.replace('[','').replace(']','').split(',')
            })       

        return result_list 

have_list = []
have = input("가진 재료 :")
while have != '끝' :
    have_list.append(have)
    have = input("가진 재료 :")
    
## 각 함수들의 리턴값인 데이터프레임형태의 레시피리스트를 concat하여 하나의 데이터프레임으로 저장
data1 = get_recipe1(have_list)
data1 = pd.DataFrame(data1)
data2 = get_recipe2(have_list)
data3 = get_recipe3(have_list)
data3 = pd.DataFrame(data3)
if len(data1['title'][0]) == 0 :
    menu = pd.concat([data2,data3])
else :
    menu = pd.concat([data1,data2,data3])
menu.reset_index(drop=True,inplace=True)
menu ## 최종 레시피 리스트

가진 재료 :두부
가진 재료 :시금치
가진 재료 :김치
가진 재료 :끝


,title,url,time,level,ingredient
0,일본식 두부 덮밥 두부 소보로동 만들기,https://www.10000recipe.com/recipe/6829153,30분 이내,중급,"[두부(부침찌개용) 300g한팩, 간장 2T, 설탕 2T(취향에따라가감), 미림 1..."
1,두부 강정..,https://www.10000recipe.com/recipe/6924603,30분 이내,아무나,"[두부 1모, 식용유 넉넉히, 전분가루 1컵, 고추장 1/2큰술, 케첩 7큰술, 간..."
2,"브로콜리 맛살 두부 무침, 브로콜리 무침, 밑반찬",https://www.10000recipe.com/recipe/6907922,15분 이내,아무나,"[브로콜리 1개(280g), 두부 1/3모, 맛살 4개(긴건2개), 맛소금 1/3큰..."
3,"초간단 아침식사, 카레향 나는 두부 달걀 스크램블 덮밥",https://www.10000recipe.com/recipe/6908353,10분 이내,아무나,"[두부 1모, 달걀 8개, 카레가루 2T, 우유 200ml, 소금 4꼬집, 후추 취..."
4,"한그릇요리, 시금치 갈릭 새우 볶음밥",https://www.10000recipe.com/recipe/6929581,10분 이내,아무나,"[시금치 1줌, 새우 1/2컵, 밥 1그릇, 식용유 3큰술, 진간장 1큰술, 다진마..."
5,두부 강정,https://www.10000recipe.com/recipe/4088595,30분 이내,초급,"[두부 1+1/2모, 녹말가루 조금, 청주 2큰술, 케찹 1큰술, 고추장 2큰술, ..."
6,두부 청경채 볶음 : 간장소스를 더해 달콤 짭조름,https://www.10000recipe.com/recipe/6877808,30분 이내,아무나,"[진간장 1스푼, 멸치액젓또는참지액 1스푼, 올리고당 2스푼, 참기름 1스푼, 다진..."
7,완전 초간단 초간편 시금치 스팸햄 볶음,https://www.10000recipe.com/recipe/6908099,10분 이내,아무나,"[스팸햄 1/2캔, 시금치 2~3포기, 청양고추 1개, 홍고추 1개, 대파 1큰술,..."
8,두부 옷을 입은 참치! 흔한 재료로 근사하게~! 참치두부전,https://www.10000recipe.com/recipe/6927016,20분 이내,초급,"[참치통조림 1개, 두부 1모, 양파 1/4개, 당근 1/4개, 전분가루 적당량, ..."
9,두부 감자조림-제철 맞은 감자와 두부의 맛 좋아요,https://www.10000recipe.com/recipe/6892112,30분 이내,초급,"[두부 1모, 감자(작) 3개, 양파(작) 1/2개, 대파 1/3뿌리, 진간장 4수..."


In [11]:
def online_market(recipe_num) :
    recipe_num = int(recipe_num)
    selected_recipe = menu.iloc[recipe_num:recipe_num+1, :]
    selected_ing = selected_recipe['ingredient'].tolist()
    selected_ing = selected_ing[0]
    
    ## 필요한 재료(선택 레시피에는 있지만 내가 가지지 못한 재료) 추출

    print("선택한 레시피의 재료들 :" ,selected_ing) 
    print("사용자가 가지고 있는 재료들 :", have_list)
    
    for i in selected_ing :
        if ('조금' in str(i)) or ('용' in str(i)):
            pass
        else :
            list1 = i.split(' ')
            a = list1[0]
            if a in have_list : 
                try :
                    b = ' '.join(list1)
                    selected_ing.remove(b)
                except :
                    pass

    no_matter = []
    for i in selected_ing :
        a = i.split(' ')
        no_matter.append(a[0])
        
    print("선택한 레시피의 재료 중에 사용자에게 없는 재료들 :", no_matter)
    
#     path = "C:\\Users\\dayoung\\Downloads\\chromedriver_win32\\chromedriver"
#     driver = webdriver.Chrome(path)
    driver = webdriver.Chrome('crawling/chromedriver.exe')
    
    ## 마켓컬리 사이트 로그인
    driver.get("https://www.kurly.com/shop/member/login.php")
    element_id = driver.find_element_by_name("m_id")
    element_pw = driver.find_element_by_name("password")
    element_id.send_keys("ldyoung")
    element_pw.send_keys("dlsrhdwlsmd1234")
    driver.find_element_by_class_name('
                                      btn_type1').click()

    element_input = driver.find_element_by_name("sword")
    element_input.clear()

    ## 검색창 접근
    for want in no_matter:
        sword = driver.find_element_by_name('sword')  # 검색창
        sword.clear()  # 검색창에 쓰여진것 지우기
        sword.send_keys(want)  # 재료 리스트 입력
        btn_search = driver.find_elements_by_class_name('btn_search')[0]
        btn_search.click()  # '검색버튼' 클릭

        # 장바구니 클릭
        driver.implicitly_wait(100000)
        xpath = driver.find_element_by_xpath('//*[@id="goodsList"]/div[1]/div/ul/li[1]/div/div/div/button')
        xpath.click()

        # 장바구니 큰 틀
        inner_option = driver.find_elements_by_class_name('inner_option')[0]

        # 장바구니 추가 버튼
        final = inner_option.find_element_by_class_name('btn_type1')

        # 옵션 이름 리스트
        name = inner_option.find_elements_by_class_name('name')   
        name_list = []
        for i in name:
            name_list.append(i.text)

        for idx, item in enumerate(name_list):
            if len(name_list) == 1:  # name_list의 길이가 1인 경우

                if '품절' not in item:
                    final.click()
                else:
                    btn_close = driver.find_element_by_class_name('btn_close1')[0]  # 장바구니 닫기
                    btn_close.click()
                    break  # 첫번째 for문으로 돌아가야 하는데 맞나?????????????????

            else: # name_list의 길이가 2 이상인 경우
                if '품절' not in item:

                    if want in item:
                        button_up = inner_option.find_elements_by_css_selector('.btn.up')[idx]  # 수량 올리기
                        button_up.click()
                        final.click()  # 장바구니 담기
                        ## 첫번째 for문으로 다시 돌아가야 하는데 맞나????????

                    else:
                        button_up = inner_option.find_elements_by_css_selector('.btn.up')[0]  # 첫번째 항목 수량 올리기
                        button_up.click()
                        final.click()  # 장바구니 P담기

                    break

                else:  # 품절이 항목에 있는 경우

                    if '품절' in item:
                        pass
                    elif want in item:
                        button_up = inner_option.find_elements_by_css_selector('.btn.up')[idx]  # 수량 올리기
                        button_up.click()
                        final.click()  # 장바구니 담기
                        break
                    else:    
                        button_up = inner_option.find_elements_by_css_selector('.btn.up')[0]  # 첫번째 항목 수량 올리기
                        button_up.click()
                        final.click()  # 장바구니 담기
                        break
                    
recipe_num = int(input("레시피 번호 선택 : "))
result = online_market(recipe_num)

레시피 번호 선택 : 13
선택한 레시피의 재료들 : ['스팸 1/2캔(2조각)', '도시락김 1장', '신김치 3줄', '쌀밥 1공기', '달걀 1개', '참기름 1T', '소금 1/2T', '통깨 1/2T']
사용자가 가지고 있는 재료들 : ['두부', '시금치', '김치']
선택한 레시피의 재료 중에 사용자에게 없는 재료들 : ['스팸', '도시락김', '신김치', '쌀밥', '달걀', '참기름', '소금', '통깨']


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=84.0.4147.105)
